In [ ]:
%pip install langdetect

In [12]:
import re
import sys
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer 

from langdetect import detect

from parapply import parapply

from sklearn.metrics import accuracy_score, classification_report

import fasttext

import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv(sys.path[0]+'/train.csv',sep=',',dtype=object)
data.shape

(497819, 4)

In [13]:
def preprocessing(data,n_jobs = 1):

    for character in string.punctuation:
        data['name_1'] = data['name_1'].apply(lambda x: str(x).replace(character, ''))
        data['name_2'] = data['name_2'].apply(lambda x: str(x).replace(character, ''))

    data['lang_1'] = parapply(data['name_1'],lambda x: detect(x),n_jobs=n_jobs)
    data['lang_2'] = parapply(data['name_2'],lambda x: detect(x),n_jobs=n_jobs)

    lang_ru_indexes =data[data['lang_1']=='ru'].index
    print(lang_ru_indexes)
    if len(list(lang_ru_indexes))==0:
        lang_other_indexes = data.index
    else:
        lang_other_indexes =data.index.drop(lang_ru_indexes)
    stemmer_ru = SnowballStemmer("russian")
    stemmer_other = SnowballStemmer("english")

    if len(list(lang_ru_indexes))!=0:
        data.loc[lang_ru_indexes,'name_1_stemmed'] = parapply(data.loc[lang_ru_indexes,'name_1'],lambda x: stemmer_ru.stem(x),n_jobs=4)
        data.loc[lang_other_indexes,'name_1_stemmed'] = parapply(data.loc[lang_other_indexes,'name_1'],lambda x: stemmer_other.stem(x),n_jobs=4)
    else:
        data.loc[lang_other_indexes,'name_1_stemmed'] = parapply(data.loc[lang_other_indexes,'name_1'],lambda x: stemmer_other.stem(x),n_jobs=4)

    lang_ru_indexes =data[data['lang_2']=='ru'].index
    print(lang_ru_indexes)
    if len(list(lang_ru_indexes))==0:
        lang_other_indexes = data.index
    else:
        lang_other_indexes =data.index.drop(lang_ru_indexes)
    stemmer_ru = SnowballStemmer("russian")
    stemmer_other = SnowballStemmer("english")

    if len(list(lang_ru_indexes))!=0:
        data.loc[lang_ru_indexes,'name_2_stemmed'] = parapply(data.loc[lang_ru_indexes,'name_2'],lambda x: stemmer_ru.stem(x),n_jobs=n_jobs)
        data.loc[lang_other_indexes,'name_2_stemmed'] = parapply(data.loc[lang_other_indexes,'name_2'],lambda x: stemmer_other.stem(x),n_jobs=n_jobs)
    else:
        data.loc[lang_other_indexes,'name_2_stemmed'] = parapply(data.loc[lang_other_indexes,'name_2'],lambda x: stemmer_other.stem(x),n_jobs=n_jobs)
   
    
    return data


def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

In [14]:
dataset = preprocessing(data,4)

In [15]:
dataset.to_csv(sys.path[0]+'/CompanyNamesSimularity_dataset.csv',sep=';',index=False)

In [17]:
dataset = pd.read_csv(sys.path[0]+'/CompanyNamesSimularity_dataset.csv',sep=';',dtype=object)

In [18]:
model_1 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=300,epoch=5, lr=0.1,thread=10)
model_2 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=150,epoch=7, lr=0.05,thread=10)
model_3 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=100,epoch=10, lr=0.01,thread=10)

Read 4M words
Number of words:  16376
Number of labels: 0
Progress: 100.0% words/sec/thread:   90969 lr:  0.000000 avg.loss:  0.554888 ETA:   0h 0m 0s  7.5% words/sec/thread:   92970 lr:  0.092471 avg.loss:  1.638084 ETA:   0h 0m23s 17.5% words/sec/thread:   91275 lr:  0.082497 avg.loss:  1.121155 ETA:   0h 0m21s 19.5% words/sec/thread:   91576 lr:  0.080489 avg.loss:  1.068077 ETA:   0h 0m20s 29.0% words/sec/thread:   92016 lr:  0.070988 avg.loss:  0.908398 ETA:   0h 0m18s 40.6% words/sec/thread:   91545 lr:  0.059436 avg.loss:  0.800276 ETA:   0h 0m15s  0h 0m11s
Read 4M words
Number of words:  16376
Number of labels: 0
Progress: 100.0% words/sec/thread:  153554 lr:  0.000000 avg.loss:  0.553253 ETA:   0h 0m 0s 53.8% words/sec/thread:  159130 lr:  0.023119 avg.loss:  0.715448 ETA:   0h 0m 9s 69.5% words/sec/thread:  156309 lr:  0.015272 avg.loss:  0.634221 ETA:   0h 0m 6s 79.9% words/sec/thread:  155292 lr:  0.010065 avg.loss:  0.599934 ETA:   0h 0m 4s 95.3% words/sec/thread:  154244 

In [5]:
model_1.save_model(sys.path[0]+'/model_1.bin')
model_2.save_model(sys.path[0]+'/model_2.bin')
model_3.save_model(sys.path[0]+'/model_3.bin')


In [19]:
model_1 = fasttext.load_model(sys.path[0]+'/model_1.bin')

In [20]:
dataset['is_duplicate'] = dataset['is_duplicate'].astype(int)

In [21]:
dataset_val = dataset[dataset['is_duplicate']==1]
dataset_val = dataset_val.reset_index(drop=True)
Y = dataset['name_2_stemmed']
print(dataset_val.shape, Y.shape)


(3658, 8) (497819,)


In [22]:
embeddings_Y = np.zeros((dataset.shape[0],300),dtype='float32')
for i,sentence in enumerate(Y):
    embeddings_Y[i,:] = model_1.get_sentence_vector(sentence)

In [23]:
dataset_val['names_1_simularity'] = None
dataset_val['min_1_distance'] = 0
results_tuple_list = []

def simularity(y,Y=Y,embeddings = embeddings_Y,model=model_1):
    distances_argmin = []
    simularity_values = []
    distances = np.zeros(embeddings.shape[0])
    vector_i = model.get_sentence_vector(y) 

    for j, sentence_j in enumerate(Y):
        vector_j = embeddings[j,:]
        distances[j] = cos_sim(vector_i,vector_j)

    distances_argmin = Y[np.argmax(distances)]
    simularity_values = np.max(distances)

    return (distances_argmin, simularity_values)

results_tuple_list.append(dataset_val['name_1_stemmed'].apply(lambda x: simularity(x)))



In [26]:
dataset_val['names_1_simularity'] = [i[0] for i in results_tuple_list[0]]
dataset_val['target'] = dataset_val['name_2_stemmed']==dataset_val['names_1_simularity']
dataset_val['target'] = dataset_val['target'].astype(int)
dataset_val['is_duplicate'] = dataset_val['is_duplicate'].astype(int)

print('accuracy_score= ',round(accuracy_score(dataset_val['is_duplicate'],dataset_val['target']),2))

accuracy_score=  0.03


In [27]:
dataset_val.head(20)


,pair_id,name_1,name_2,is_duplicate,lang_1,lang_2,name_1_stemmed,name_2_stemmed,names_1_simularity,min_1_distance,target
0,162,JX Nippon Oil Gas Exploration Brasil Ltda,JX Nippon Oil Gas Exploration Technical Servi...,1,en,en,jx nippon oil gas exploration brasil ltda,jx nippon oil gas exploration technical servi...,jx nippon oil gas exploration brasil ltda,0,0
1,604,Pirelli Neumaticos SAIC,Pirelli Tyre Co Ltd,1,it,it,pirelli neumaticos sa,pirelli tyre co ltd,pirelli neumaticos sa,0,0
2,836,Brenntag Australia Pty Ltd,Brenntag Group,1,en,de,brenntag australia pty ltd,brenntag group,brenntag australia pty ltd,0,0
3,1329,PAUL BAUDER GMBH CO KG BOCHUM PLANT,Paul Bauder ag,1,de,de,paul bauder gmbh co kg bochum pl,paul bauder ag,paul bauder gmbh co kg bochum pl,0,0
4,1563,TOTAL CESKA REPUBLIKA sro,TOTAL FRANCE ARNAY LE DUC,1,en,en,total ceska republika sro,total france arnay le duc,total ceska republika sro,0,0
5,1787,Pt Bridgestone Tire Indonesia,Bridgestone Do Brasil Industria E Comenrcio Ltda,1,en,it,pt bridgestone tire indonesia,bridgestone do brasil industria e comenrcio ltda,pt bridgestone tire indonesia,0,0
6,1799,Brenntag Peru SAC,Brenntag Group,1,de,de,brenntag peru sac,brenntag group,brenntag peru sac,0,0
7,1963,Carlisle Coatings Waterproofing Inc,Carlisle Coatings Wtrprfng,1,en,en,carlisle coatings waterproofing inc,carlisle coatings wtrprfng,carlisle coatings waterproofing inc,0,0
8,2055,Pt Bridgestone Tire Indonesia,Thai Bridgestone Co Ltd,1,en,en,pt bridgestone tire indonesia,thai bridgestone co ltd,pt bridgestone tire indonesia,0,0
9,2773,Onduline Polska Sp z oo,Onduline SA,1,pl,et,onduline polska sp z oo,onduline sa,swisspor polska sp z oo,0,0


In [28]:
dataset_val[dataset_val['target']==1]

,pair_id,name_1,name_2,is_duplicate,lang_1,lang_2,name_1_stemmed,name_2_stemmed,names_1_simularity,min_1_distance,target
26,4647,Coim SpA,COIM SpA,1,de,en,coim spa,coim spa,coim spa,0,1
28,5118,ОООИМПОРТ МОДА,ОООИМПОРТ МОДА,1,bg,bg,оооимпорт мода,оооимпорт мода,оооимпорт мода,0,1
40,6453,Midland Asphalt Materials Inc,Midland Asphalt,1,en,en,midland asphalt materials inc,midland asphalt,midland asphalt,0,1
82,12161,ОООГУЧЧИ РУС,ОООГУЧЧИ РУС,1,bg,bg,ооогуччи рус,ооогуччи рус,ооогуччи рус,0,1
104,14858,Lohmann GmbH Co KG,Lohmann GmbH Co KG,1,de,de,lohmann gmbh co kg,lohmann gmbh co kg,lohmann gmbh co kg,0,1
...,...,...,...,...,...,...,...,...,...,...,...
3508,476491,ООООРТГРАФ,ООООРТГРАФ,1,bg,bg,оооортграф,оооортграф,оооортграф,0,1
3540,479648,ООО КЬЮ ЭКСПРЕСС,ООО КЬЮ ЭКСПРЕСС,1,ru,ru,ооо кью экспресс,ооо кью экспресс,ооо кью экспресс,0,1
3555,481656,ООО БРААСДСК 1,ООО БРААСДСК 1,1,ru,ru,ооо браасдск 1,ооо браасдск 1,ооо браасдск 1,0,1
3604,488072,ООО МАНДЕРС,ООО МАНДЕРС,1,ru,ru,ооо мандерс,ооо мандерс,ооо мандерс,0,1
